## Baseline UNet Code

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf 
from tensorflow.keras.layers import * 
from tensorflow.keras import Model, Input  
from tensorflow.keras.models import * 
from tensorflow.keras.callbacks import *
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split 
from sklearn.metrics import f1_score
import random 
import os 
import time 
from tqdm import tqdm 
import keras.backend as K

## Load Dataset

In [2]:
train = pd.read_csv('train.csv') 
filenames = train['file_nm'].values

In [3]:
X_train = []  
y_train = [] 

# use the past 3 timesteps to predict the next 12 timesteps 
past = 60
forecast = 24
for i in tqdm(range(len(filenames)-past-forecast), position = 0, leave = True): 
    X_train_sample = np.load('train/train/'+filenames[i]) 
    X_train_sample = (X_train_sample[:,:,0]/250 + X_train_sample[:,:,1] + X_train_sample[:,:,2]).reshape((448,304,1)) 
    
    y_train_sample = np.load('train/train/'+filenames[i+past])
    y_train_sample = (y_train_sample[:,:,0]/250 + y_train_sample[:,:,1] + y_train_sample[:,:,2]).reshape((448,304,1)) 
    
    for j in range(i+1, i+past+forecast):  
        sample = np.load('train/train/' + filenames[j]) 
        image = (sample[:,:,0]/250 + sample[:,:,1] + sample[:,:,2]).reshape((448,304,1))
        if j < i+past: 
            X_train_sample = np.concatenate([X_train_sample, image], axis = -1) 
        elif j > i+past: 
            y_train_sample = np.concatenate([y_train_sample, image], axis = -1) 
    
    X_train.append(X_train_sample) 
    y_train.append(y_train_sample) 
    
X_train = np.asarray(X_train) 
y_train = np.asarray(y_train) 

X_train.shape, y_train.shape


100%|██████████| 398/398 [05:53<00:00,  1.14it/s]


((398, 448, 304, 60), (398, 448, 304, 24))

## Define Loss Function

In [4]:
def dice_loss(y_true, y_pred): 
    y_true = tf.cast(y_true, tf.float32) 
    y_pred = tf.cast(y_pred, tf.float32) 
    numerator = 2*tf.reduce_sum(y_true * y_pred) 
    denominator = tf.reduce_sum(y_true + y_pred + 1e-2)     
    return numerator / denominator 

def custom_loss(y_true, y_pred): 
    y_true = tf.cast(y_true, tf.float32) 
    y_pred = tf.cast(y_pred, tf.float32) 
    mae = tf.reduce_mean(tf.abs(y_true - y_pred)) 
    dice = dice_loss(y_true, y_pred) 
    return mae / dice 

## Define Model

In [5]:
def UNet(input_layer, start_neurons = 32): 
    bn = BatchNormalization()(input_layer) # preprocessing step 
    conv1 = Conv2D(start_neurons * 1, (5, 5), activation = 'relu', padding = 'same')(bn) 
    conv1 = BatchNormalization()(conv1) 
    pool1 = MaxPooling2D((2,2))(conv1) 
    pool1 = Dropout(0.25)(pool1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation = 'relu', padding = 'same')(pool1)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2,2))(conv2)  
    pool2 = Dropout(0.25)(pool2)

    conv3 = Conv2D(start_neurons * 4, (3, 3), activation = 'relu', padding = 'same')(pool2) 
    conv3 = BatchNormalization()(conv3) 
    pool3 = MaxPooling2D((2,2))(conv3) 
    pool3 = Dropout(0.25)(pool3)  

    convm = Conv2D(start_neurons * 8, (3, 3), activation = 'relu', padding = 'same')(pool3) 

    deconv3 = Conv2DTranspose(start_neurons * 4, (3,3), strides = (2,2), activation = 'relu', padding = 'same')(convm) 
    uconv3 = concatenate([deconv3, conv3]) 
    uconv3 = Conv2D(start_neurons * 4, (3,3), activation = 'relu', padding = 'same')(uconv3) 
    uconv3 = BatchNormalization()(uconv3) 
    uconv3 = Dropout(0.25)(uconv3) 

    deconv2 = Conv2DTranspose(start_neurons * 2, (3,3), strides = (2,2), activation = 'relu', padding = 'same')(uconv3) 
    uconv2 = concatenate([deconv2, conv2]) 
    uconv2 = Conv2D(start_neurons * 2, (3,3), activation = 'relu', padding = 'same')(uconv2) 
    uconv2 = BatchNormalization()(uconv2) 
    uconv2 = Dropout(0.25)(uconv2) 

    deconv1 = Conv2DTranspose(start_neurons * 1, (3,3), strides = (2,2), activation = 'relu',  padding = 'same')(uconv2)
    uconv1 = concatenate([deconv1, conv1]) 
    uconv1 = Conv2D(start_neurons * 1, (3,3), activation = 'relu', padding = 'same')(uconv1) 
    uconv1 = BatchNormalization()(uconv1) 
    uconv1 = Dropout(0.25)(uconv1) 

    outputs = Conv2D(24, (1,1), activation = 'relu', padding = 'same')(uconv1) 

    model = Model(inputs = input_layer, outputs = outputs)

    model.compile(loss = custom_loss, optimizer = 'adam')
    return model 

## Train

In [6]:
input_layer = Input((448, 304, 60)) 
model = UNet(input_layer)

model_path = 'models/epoch_{epoch:03d}_val_{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath=model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
learning_rate_reduction = ReduceLROnPlateau(moniitor = 'val_loss', patience = 1, verbose = 1, factor = 0.8)
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5) 
history = model.fit(X_train,
                    y_train,
                    epochs = 200,
                    batch_size = 32,  
                    validation_split = 0.1, 
                    callbacks = [learning_rate_reduction, checkpoint, early_stopping])

Epoch 1/200
12/12 [==============================] - 44s 1s/step - loss: 0.6793 - val_loss: 0.5677

Epoch 00001: val_loss improved from inf to 0.56767, saving model to models/epoch_001_val_0.568.h5
Epoch 2/200
12/12 [==============================] - 8s 661ms/step - loss: 0.3061 - val_loss: 0.2849

Epoch 00002: val_loss improved from 0.56767 to 0.28492, saving model to models/epoch_002_val_0.285.h5
Epoch 3/200
12/12 [==============================] - 9s 781ms/step - loss: 0.2197 - val_loss: 0.1531

Epoch 00003: val_loss improved from 0.28492 to 0.15312, saving model to models/epoch_003_val_0.153.h5
Epoch 4/200
12/12 [==============================] - 16s 1s/step - loss: 0.1608 - val_loss: 0.1513

Epoch 00004: val_loss improved from 0.15312 to 0.15130, saving model to models/epoch_004_val_0.151.h5
Epoch 5/200
12/12 [==============================] - 18s 2s/step - loss: 0.1339 - val_loss: 0.4046

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.000800000037997961.

Epoch 00005:

12/12 [==============================] - 14s 1s/step - loss: 0.0695 - val_loss: 0.0378

Epoch 00040: val_loss improved from 0.03809 to 0.03785, saving model to models/epoch_040_val_0.038.h5
Epoch 41/200
12/12 [==============================] - 15s 1s/step - loss: 0.0696 - val_loss: 0.0373

Epoch 00041: val_loss improved from 0.03785 to 0.03727, saving model to models/epoch_041_val_0.037.h5
Epoch 42/200
12/12 [==============================] - 13s 1s/step - loss: 0.0699 - val_loss: 0.0384

Epoch 00042: ReduceLROnPlateau reducing learning rate to 5.497558740898967e-05.

Epoch 00042: val_loss did not improve from 0.03727
Epoch 43/200
12/12 [==============================] - 14s 1s/step - loss: 0.0706 - val_loss: 0.0387

Epoch 00043: ReduceLROnPlateau reducing learning rate to 4.398046876303852e-05.

Epoch 00043: val_loss did not improve from 0.03727
Epoch 44/200
12/12 [==============================] - 15s 1s/step - loss: 0.0703 - val_loss: 0.0382

Epoch 00044: ReduceLROnPlateau reducing 

KeyboardInterrupt: 

## Inference

In [7]:
best_model = load_model('models/epoch_051_val_0.037.h5', custom_objects = {'custom_loss':custom_loss})   

In [8]:
# check model summary 
best_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 448, 304, 60 0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 448, 304, 60) 240         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 448, 304, 32) 48032       batch_normalization[0][0]        
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 448, 304, 32) 128         conv2d[0][0]                     
______________________________________________________________________________________________

In [25]:
X_test = np.load('train/train/' + filenames[len(filenames)-60])[:,:,0].reshape((448,304,1)) 

for i in tqdm(range(len(filenames)-59, len(filenames)), position = 0, leave = True): 
    sample = np.load('train/train/' + filenames[i])
    image = sample[:,:,0].reshape((448,304,1)) 
    X_test = np.concatenate([X_test, image], axis = -1) 
    

X_test = np.asarray(X_test) 
X_test.shape

100%|██████████| 59/59 [00:00<00:00, 453.44it/s]


(448, 304, 60)

In [26]:
## multi step prediction 
X_test = X_test.reshape((1,448,304,60))
preds = best_model.predict(X_test) # first prediction  

In [34]:
preds *= 250.0

In [35]:
preds = preds.reshape((24,448*304))
preds.shape

(24, 136192)

In [36]:
sample_submission = pd.read_csv('sample_submission.csv') 
sample_submission.iloc[:,1:] = preds

In [37]:
sample_submission.to_csv('UNet_3.csv',index=False)